In [1]:
import numpy as np
import pandas as pd
from utils import di_ratios,dictionaries,segment_mapper

BMM adat és a takeup adat közötti kapcsolat megteremtése a product mapping kapcsán

In [255]:
bmm_data=pd.read_csv(filepath_or_buffer="../data/output_3.csv")
bmm_data.columns=bmm_data.columns.str.lower().str.replace(" ","_")
bmm_data["s_level_segment"]=bmm_data["s_level_segment"].str.upper()
bmm_data.sample(n=1)

,country,level_1,level_2,level_3,level_4,segment,core_market,orig_ccy,orig_ccy_fy23,orig_ccy_fy24,...,fy26,fy27,fy28,region,s_level_segment,#_business_2021,#_business_2022,#_business_2023,avg_spend_23,avg_spend_24
7055,United Kingdom,Fixed,Fixed Data (SD),Ethernet (SD),Ethernet - E-LAN (SD),Large Corporate (1000+),Core,GBP,1.714395,3.434711,...,10.974714,16.075898,22.44043,EU2,S9-S11,NaN,NaN,NaN,NaN,NaN


In [256]:
take_up_sc=pd.read_excel(
    io="../data/Data Landscaping Master (Draft) - PQ WIP.xlsx",
    sheet_name="Take-up Now Append",
    skiprows=6
)
take_up_sc.columns=take_up_sc.columns.str.lower().str.replace(" ","_")
take_up=take_up_sc.loc[:,~take_up_sc.columns.str.contains(pat="(sc)",regex=False)]
take_up.sample(n=1)

,source,product,bmm_match,maturity,size_class,de,pt,uk,nl,ie,sa,tr,cz,gr,ro,al
582,Omdia SoHo SME,"UC&C apps (e.g., Microsoft Teams, Zoom, WebEx,...",Conferencing Application - On Premise,Mature,SME 1 (10-49),0.780858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [257]:
s_level_segment=take_up["size_class"].apply(
    lambda size_class: dictionaries["takeup_segments"].get(size_class,np.nan)
).values
take_up.loc[:,"s_level_segment"]=s_level_segment
take_up.rename(
    columns={
        "de":"Germany",
        "pt":"Portugal",
        "uk":"United Kingdom",
        "nl":"Netherlands",
        "ie":"Ireland",
        "sa":"South Africa",
        "tr":"Turkey",
        "cz":"Czechia",
        "gr":"Greece",
        "al":"Albania",
        "ro":"Romania"
    },
    inplace=True
)
take_up.sample(n=1)

C:\Users\BiroB\AppData\Local\Temp\ipykernel_2528\4160730927.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  take_up.loc[:,"s_level_segment"]=s_level_segment
C:\Users\BiroB\AppData\Local\Temp\ipykernel_2528\4160730927.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  take_up.rename(


,source,product,bmm_match,maturity,size_class,Germany,Portugal,United Kingdom,Netherlands,Ireland,South Africa,Turkey,Czechia,Greece,Romania,Albania,s_level_segment
552,VF Cybersecurity survey,SSO and MFA,NaN,NaN,25-99,0.230769,0.32,0.38961,NaN,0.204545,NaN,NaN,NaN,NaN,NaN,NaN,S5


In [258]:
# a level_3_copy a BMM-es adat
product_mapper=pd.read_csv(filepath_or_buffer="../data/product_mapper.csv")
product_mapper.columns=product_mapper.columns.str.lower().str.replace(" ","_")
product_mapper.sample(n=1)

,level_3_copy,product,"level_match_(1,_2_or_3)",maturity
40,Team Collaboration - Cloud,Collaboration applications,2.0,Growing


In [259]:
# take up with products
tu_w_prods=take_up.drop(columns=["maturity"]).merge(
    right=product_mapper,
    how="left",
    left_on=["product"],
    right_on=["product"]
).dropna(subset=["level_3_copy"])
tu_w_prods.rename(columns={"level_3_copy":"level_3"},inplace=True)
tu_w_prods["s_level_segment"]=tu_w_prods["s_level_segment"].str.upper()
tu_w_prods.sample(n=1)

,source,product,bmm_match,size_class,Germany,Portugal,United Kingdom,Netherlands,Ireland,South Africa,Turkey,Czechia,Greece,Romania,Albania,s_level_segment,level_3,"level_match_(1,_2_or_3)",maturity
300,Omdia SoHo SME Sec,Identity and Access Security - strengthening p...,Identity & Digital Trust,5 to 9,0.416667,NaN,0.529412,NaN,NaN,0.5625,NaN,NaN,NaN,NaN,NaN,S3,Identity & Digital Trust,3.0,Growing


In [260]:
transform_table=pd.read_excel(io="../data/transform_table.xlsx")

In [270]:
# add more granular segments to take up data
tu_granular_segments=pd.DataFrame(
    data=tu_w_prods["s_level_segment"].apply(segment_mapper,args=(transform_table,)).tolist(),
    columns=["segment_exact_match","s_lvl_closest_match"]
)
tup_w_segments=pd.concat(
    objs=[tu_w_prods.reset_index(drop=True),tu_granular_segments],
    axis=1
)
tup_w_segments.sample(n=3)

,source,product,bmm_match,size_class,Germany,Portugal,United Kingdom,Netherlands,Ireland,South Africa,...,Czechia,Greece,Romania,Albania,s_level_segment,level_3,"level_match_(1,_2_or_3)",maturity,segment_exact_match,s_lvl_closest_match
530,FFTF survey,Mobile private networks (MPN),Fully Dedicated-Radio Access Network,Small Business: 10-49,0.315789,0.304348,0.378049,0.305556,NaN,NaN,...,NaN,NaN,NaN,NaN,S4,Fully Dedicated-Core,1.0,Emerging,True,S3-S4;S4;S4-S6
785,P&S Team,Public Cloud,Public Cloud (IaaS Resell),Mid-Market (250-999),0.770000,0.770000,0.820000,0.820000,0.840000,NaN,...,0.77,0.54,0.42,0.47,S7-S8,Public Cloud (IaaS Resell),3.0,Growing,False,S7-S8
313,VF Cybersecurity survey,Managed detection and response services (MDR),MDR,0,0.046875,0.596154,0.088235,NaN,0.447368,NaN,...,NaN,NaN,NaN,NaN,S1,MDR,3.0,Growing,False,S1


In [271]:
# add more granular segments to bmm
bmm_data_segments=pd.DataFrame(
    data=bmm_data["s_level_segment"].apply(segment_mapper,args=(transform_table,)).tolist(),
    columns=["segment_exact_match","s_lvl_closest_match"]
)
bmm_wsegments=pd.concat(
    objs=[bmm_data,bmm_data_segments],
    axis=1
)
bmm_wsegments.sample(n=3)

,country,level_1,level_2,level_3,level_4,segment,core_market,orig_ccy,orig_ccy_fy23,orig_ccy_fy24,...,fy28,region,s_level_segment,#_business_2021,#_business_2022,#_business_2023,avg_spend_23,avg_spend_24,segment_exact_match,s_lvl_closest_match
3817,Netherlands,Fixed,LAN/WLAN Equipment and Services,LAN/WLAN Maintenance Services,LAN Equipment - Maintenance Services - Deploy ...,Small Corporate (100-249),NaN,EUR,7.680179,7.538510,...,7.547115,Non-Group,S6,6652.634708,6764.952737,6942.894782,0.001135,0.001086,True,S5-S7;S6
7488,United Kingdom,Security,Security - Managed Security Services,"Analytics, Intelligence, Response and Orchestr...","Analytics, Intelligence, Response and Orchestr...",Large Corporate (250-499),Core,GBP,2.167772,2.402225,...,4.417731,EU2,S7,4192.300209,4206.221510,4339.468243,0.000586,0.000629,False,S5-S7;S7-S8
7453,United Kingdom,Mobile Private Networks,Hybrid Network,Hybrid Network-Professional Services,Hybrid Network-Professional Services,Large Corporate (250-499),Not offered,GBP,NaN,0.000000,...,0.445459,EU2,S7,4192.300209,4206.221510,4339.468243,NaN,0.000000,False,S5-S7;S7-S8


In [279]:
bmm_wsegments[["s_lvl_closest_match","level_3"]].merge(
    right=tup_w_segments[["s_lvl_closest_match","level_3","maturity"]],
    how="left",
    left_on=["s_lvl_closest_match","level_3"],
    right_on=["s_lvl_closest_match","level_3"]
).drop_duplicates(subset=["s_lvl_closest_match","level_3","maturity"]).dropna(subset=["maturity"])

,s_lvl_closest_match,level_3,maturity
0,S1;S1-S3;S2;S2-S3;S3;S3-S4,Cloud Connectivity (non-SD),Growing
8,S3-S4;S4;S4-S6,Ethernet (non-SD),Mature
15,S7-S11;S9-S11,Ethernet (non-SD),Mature
41,S3-S4;S4;S4-S6,Internet Access (non-SD),Mature
83,S3-S4;S4;S4-S6,Leased Lines (non-SD),Mature
...,...,...,...
2812,S1;S1-S3;S2;S2-S3;S3;S3-S4,DaaS - Service - Smartphone,Mature
2826,S1;S1-S3;S2;S2-S3;S3;S3-S4,DaaS - Service - Tablet,Mature
3148,S3-S4;S4;S4-S6,SaaS - Fileshare & Synch,Growing
3149,S5-S7;S6,SaaS - Fileshare & Synch,Growing


In [282]:
bmm_wsegments["level_3"].isin(tup_w_segments["level_3"].values).value_counts()

level_3
True     6832
False     973
Name: count, dtype: int64

In [283]:
di_ratios

,country,mature_product,growing_product,emerging_product
0,Germany,1.000000,1.000000,1.000000
1,United Kingdom,1.092271,1.137255,1.207664
2,Turkey,0.778935,0.671160,0.502471
3,Albania,0.660296,0.494682,0.235463
4,South Africa,0.731878,0.601163,0.396566
5,Czechia,0.892436,0.860946,0.753099
6,Ireland,0.920755,0.936423,1.159091
7,Greece,0.671345,0.511118,0.327479
8,Netherlands,1.011800,1.057626,1.253099
9,Portugal,0.928953,0.947072,0.810950


In [459]:
def get_takeup(row:pd.Series)->float:
    country,product,segment=row["country"],row["level_3"],row["s_lvl_closest_match"]
    subdf=tup_w_segments[
        (tup_w_segments["level_3"]==product)
        &(tup_w_segments["s_lvl_closest_match"]==segment)
    ]
    if subdf.shape[0]==0:
        if product in tup_w_segments["level_3"].values:
            maturity=tup_w_segments[tup_w_segments["level_3"]==product]["maturity"].values[0]
            takeup=di_ratios[di_ratios["country"]==country][f"{maturity.lower()}_product"].values[0]
        else:
            takeup=np.mean(trial[7655].drop(columns=["country"]).reset_index(drop=True).T[0].values)
        return [np.nan,takeup]
    else:
        return [subdf[country].values[0],np.nan]

In [461]:
turs=pd.DataFrame(
    data=bmm_wsegments.apply(get_takeup,axis=1).tolist(),
    columns=["tur","tur_sc"]
)


In [467]:
bmm_wsegments[turs.isna().sum(axis=1)==2]

,country,level_1,level_2,level_3,level_4,segment,core_market,orig_ccy,orig_ccy_fy23,orig_ccy_fy24,...,fy28,region,s_level_segment,#_business_2021,#_business_2022,#_business_2023,avg_spend_23,avg_spend_24,segment_exact_match,s_lvl_closest_match
0,Albania,Fixed,Fixed Data (non-SD),Cloud Connectivity (non-SD),Cloud Connectivity (non-SD),SoHo (0-9),Core,ALL,0.000000,0.000000,...,0.000000,Cluster,S1-S3,97022.000000,106132.000000,116097.394653,0.000000,0.000000,True,S1;S1-S3;S2;S2-S3;S3;S3-S4
8,Albania,Fixed,Fixed Data (non-SD),Ethernet (non-SD),Ethernet - E-LAN (non-SD),Small SME (10-49),Core,ALL,5.838062,6.168127,...,0.069505,Cluster,S4,5608.000000,5841.000000,6084.400126,0.000010,0.000010,True,S3-S4;S4;S4-S6
13,Albania,Fixed,Fixed Data (non-SD),Ethernet (non-SD),Ethernet - E-LAN (non-SD),Large Corporate (1000+),Core,ALL,2.854361,2.871816,...,0.026913,Cluster,S9-S11,NaN,NaN,NaN,NaN,NaN,True,S7-S11;S9-S11
15,Albania,Fixed,Fixed Data (non-SD),Ethernet (non-SD),Ethernet - E-Line (non-SD),Small SME (10-49),Core,ALL,50.473964,51.824928,...,0.523090,Cluster,S4,5608.000000,5841.000000,6084.400126,0.000086,0.000085,True,S3-S4;S4;S4-S6
20,Albania,Fixed,Fixed Data (non-SD),Ethernet (non-SD),Ethernet - E-Line (non-SD),Large Corporate (1000+),Core,ALL,24.684820,24.135000,...,0.202591,Cluster,S9-S11,NaN,NaN,NaN,NaN,NaN,True,S7-S11;S9-S11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7765,United Kingdom,Unified Comms and Collaboration,UC Collaboration,Conferencing Application - Cloud,Conferencing Application - Cloud,Small Corporate (100-249),Core,GBP,37.501743,43.632229,...,81.726612,EU2,S6,26284.311748,26388.426775,27003.651933,0.001615,0.001836,True,S5-S7;S6
7766,United Kingdom,Unified Comms and Collaboration,UC Collaboration,Conferencing Application - Cloud,Conferencing Application - Cloud,Large SME (50-99),Core,GBP,25.001162,29.088152,...,54.484408,EU2,S5,15370.688252,15431.573225,15791.348067,0.001841,0.002093,True,S4-S6;S5;S5-S6;S5-S7
7771,United Kingdom,Unified Comms and Collaboration,UC Collaboration,Conferencing Application - On Premise,Conferencing Application - On Premise,Small SME (10-49),Core,GBP,1.671312,1.441875,...,0.683779,EU2,S4,232815.000000,239095.000000,244240.000000,0.000008,0.000007,True,S3-S4;S4;S4-S6
7772,United Kingdom,Unified Comms and Collaboration,UC Collaboration,Conferencing Application - On Premise,Conferencing Application - On Premise,Small Corporate (100-249),Core,GBP,1.323130,1.131141,...,0.451452,EU2,S6,26284.311748,26388.426775,27003.651933,0.000057,0.000048,True,S5-S7;S6


In [60]:
results=pd.concat(
    objs=[take_up,mapped_segments],
    axis=1
)

In [61]:
product_mapper=pd.read_csv("../data/product_mapper.csv")
product_mapper.columns=product_mapper.columns.str.lower().str.replace(" ","_")
product_mapper.head(n=3)

,level_3_copy,product,"level_match_(1,_2_or_3)",maturity
0,Cloud Connectivity (non-SD),Cloud Connect,3.0,Growing
1,Ethernet (non-SD),"WAN (e.g. IPVPN, Ethernet)",3.0,Mature
2,Internet Access (non-SD),Fixed broadband,2.0,Mature


In [63]:
output_4=input_3.join(
    other=product_mapper.set_index("level_3_copy"),
    on="level_3"
)
output_4.head(n=3)

,country,level_1,level_2,level_3,level_4,segment,core_market,orig_ccy,orig_ccy_fy23,orig_ccy_fy24,...,region,s_level_segment,#_business_2021,#_business_2022,#_business_2023,avg_spend_23,avg_spend_24,product,"level_match_(1,_2_or_3)",maturity
0,Albania,Fixed,Fixed Data (non-SD),Cloud Connectivity (non-SD),Cloud Connectivity (non-SD),SoHo (0-9),Core,ALL,0.000000,0.000000,...,Cluster,s1-s3,97022.000000,106132.000000,116097.394653,0.000000,0.000000,Cloud Connect,3.0,Growing
1,Albania,Fixed,Fixed Data (non-SD),Cloud Connectivity (non-SD),Cloud Connectivity (non-SD),Small SME (10-49),Core,ALL,10.619282,12.700880,...,Cluster,s4,5608.000000,5841.000000,6084.400126,0.000018,0.000021,Cloud Connect,3.0,Growing
2,Albania,Fixed,Fixed Data (non-SD),Cloud Connectivity (non-SD),Cloud Connectivity (non-SD),Small Corporate (100-249),Core,ALL,15.500858,17.718191,...,Cluster,s6,774.237199,824.086212,877.144737,0.000188,0.000202,Cloud Connect,3.0,Growing


In [65]:
take_up

,source,product,bmm_match,maturity,size_class,de,pt,uk,nl,ie,sa,tr,cz,gr,ro,al,segment_match
0,VF Cybersecurity survey,24/7 security monitoring,NaN,NaN,100-299,0.295455,0.444444,0.506024,NaN,0.400000,NaN,NaN,NaN,NaN,NaN,NaN,S5-S7
1,VF Cybersecurity survey,24/7 security monitoring,NaN,NaN,25-99,0.384615,0.340000,0.389610,NaN,0.340909,NaN,NaN,NaN,NaN,NaN,NaN,S5
2,VF Cybersecurity survey,24/7 security monitoring,NaN,NaN,5-24,0.411765,0.240000,0.354430,NaN,0.625000,NaN,NaN,NaN,NaN,NaN,NaN,S3-S4
3,VF Cybersecurity survey,24/7 security monitoring,NaN,NaN,1-4,0.185185,0.212766,0.137500,NaN,0.275000,NaN,NaN,NaN,NaN,NaN,NaN,S2
4,VF Cybersecurity survey,24/7 security monitoring,NaN,NaN,0,0.125000,0.346154,0.191176,NaN,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,S1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,Omdia SoHo SME Sec,Website and Application security – protection ...,NaN,NaN,2 to 4,0.466667,NaN,0.333333,NaN,NaN,0.363636,NaN,NaN,NaN,NaN,NaN,S2
651,Omdia SoHo SME Sec,Website and Application security – protection ...,NaN,NaN,5 to 9,0.666667,NaN,0.470588,NaN,NaN,0.562500,NaN,NaN,NaN,NaN,NaN,S3
652,Omdia SoHo SME Sec,Website and Application security – protection ...,NaN,NaN,10 to 49,0.525000,NaN,0.459459,NaN,NaN,0.605263,NaN,NaN,NaN,NaN,NaN,S4
653,Omdia SoHo SME Sec,Website and Application security – protection ...,NaN,NaN,50 to 99,0.357143,NaN,0.588235,NaN,NaN,0.625000,NaN,NaN,NaN,NaN,NaN,S5


In [37]:
def di_mapping(row:pd.Series)->float:
    di_selection=di_ratios[di_ratios["country"]==row["country"]]
    maturity=row["maturity"].lower()
    try:
        return di_selection[f"{maturity}_product"].values[0]
    except:
        return (row["country"],maturity)

In [15]:
output_4["di_ratio"]=output_4.apply(di_mapping,axis=1)

In [16]:
output_4.to_csv("../data/output_4.csv",index=False)

In [17]:
output_4

,country,level_1,level_2,level_3,level_4,segment,core_market,orig_ccy,orig_ccy_fy23,orig_ccy_fy24,...,s_level_segment,#_business_2021,#_business_2022,#_business_2023,avg_spend_23,avg_spend_24,product,"level_match_(1,_2_or_3)",maturity,di_ratio
0,Albania,Fixed,Fixed Data (non-SD),Cloud Connectivity (non-SD),Cloud Connectivity (non-SD),SoHo (0-9),Core,ALL,0.000000,0.000000,...,s1-s3,97022.000000,106132.000000,116097.394653,0.000000,0.000000,Cloud Connect,3.0,Growing,0.494682
1,Albania,Fixed,Fixed Data (non-SD),Cloud Connectivity (non-SD),Cloud Connectivity (non-SD),Small SME (10-49),Core,ALL,10.619282,12.700880,...,s4,5608.000000,5841.000000,6084.400126,0.000018,0.000021,Cloud Connect,3.0,Growing,0.494682
2,Albania,Fixed,Fixed Data (non-SD),Cloud Connectivity (non-SD),Cloud Connectivity (non-SD),Small Corporate (100-249),Core,ALL,15.500858,17.718191,...,s6,774.237199,824.086212,877.144737,0.000188,0.000202,Cloud Connect,3.0,Growing,0.494682
3,Albania,Fixed,Fixed Data (non-SD),Cloud Connectivity (non-SD),Cloud Connectivity (non-SD),Large SME (50-99),Core,ALL,10.333905,11.812127,...,s5,452.762801,481.913788,512.941653,0.000215,0.000230,Cloud Connect,3.0,Growing,0.494682
4,Albania,Fixed,Fixed Data (non-SD),Cloud Connectivity (non-SD),Cloud Connectivity (non-SD),Large Corporate (500-999),Core,ALL,18.750615,21.647297,...,s8,46.262534,48.123671,50.059680,0.003898,0.004326,Cloud Connect,3.0,Growing,0.494682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7800,United Kingdom,Unified Comms and Collaboration,UC&C Life Cycle Services,UCC Support and Maintenance Services,UCC Support and Maintenance Services,Small Corporate (100-249),Core,GBP,7.266739,7.704166,...,s6,26284.311748,26388.426775,27003.651933,0.000313,0.000324,NaN,NaN,Growing,1.137255
7801,United Kingdom,Unified Comms and Collaboration,UC&C Life Cycle Services,UCC Support and Maintenance Services,UCC Support and Maintenance Services,Large SME (50-99),Core,GBP,4.844493,5.136110,...,s5,15370.688252,15431.573225,15791.348067,0.000357,0.000370,NaN,NaN,Growing,1.137255
7802,United Kingdom,Unified Comms and Collaboration,UC&C Life Cycle Services,UCC Support and Maintenance Services,UCC Support and Maintenance Services,Large Corporate (500-999),Core,GBP,23.882154,25.350059,...,s8,2802.339712,2811.645394,2900.714066,0.009652,0.009931,NaN,NaN,Growing,1.137255
7803,United Kingdom,Unified Comms and Collaboration,UC&C Life Cycle Services,UCC Support and Maintenance Services,UCC Support and Maintenance Services,Large Corporate (250-499),Core,GBP,23.108676,24.506162,...,s7,4192.300209,4206.221510,4339.468243,0.006243,0.006417,NaN,NaN,Growing,1.137255


In [20]:
output_4["maturity"].unique()

array(['Growing', 'Mature', 'Emerging'], dtype=object)

In [21]:
product_mapper.sample(n=5)

,level_3_copy,product,"level_match_(1,_2_or_3)",maturity
89,DaaS - Service - Smartphone,DaaS / Device Services,NaN,Mature
75,SaaS - Quotes & Invoicing,NaN,NaN,Growing
66,SaaS - Business Intelligence (BI),Business intelligence (BI) systems that utiliz...,3.0,Growing
84,Remotely Managed - SaaS,SaaS Support,2.0,Growing
48,Managed Services (Private Cloud),Managed Services (Private Cl.),3.0,Growing


In [22]:
output_4.sample(n=4)

,country,level_1,level_2,level_3,level_4,segment,core_market,orig_ccy,orig_ccy_fy23,orig_ccy_fy24,...,s_level_segment,#_business_2021,#_business_2022,#_business_2023,avg_spend_23,avg_spend_24,product,"level_match_(1,_2_or_3)",maturity,di_ratio
7643,United Kingdom,Software as a Service (SaaS),SaaS - Functional Applications,SaaS - Marketing Automation,SaaS - Marketing Automation,Large Corporate (1000+),Not offered,GBP,78.445348,91.210819,...,s9-s11,NaN,NaN,NaN,NaN,NaN,"Marketing automation (e.g., Mailchimp)",3.0,Growing,1.137255
4232,Portugal,Fixed,Fixed Data (non-SD),Private IP (IP VPN) (non-SD),Private IP (IP VPN) (non-SD),Large Corporate (500-999),Core,EUR,9.720417,8.456780,...,s8,2.858174e+02,3.001747e+02,3.264965e+02,3.238253e-02,2.590159e-02,"WAN (e.g. IPVPN, Ethernet)",3.0,Mature,0.928953
6881,United Kingdom,Cloud & Hosting,Cloud Infrastructure and Services,Cloud Professional Services (Assessment),Cloud Professional Services (Assessment),SoHo (0-9),Core,GBP,1.118347,1.282783,...,s1-s3,2.480140e+06,2.476210e+06,2.428885e+06,5.132232e-07,6.001552e-07,Cloud Prof Services (Assess),3.0,Growing,1.137255
5485,South Africa,Cloud & Hosting,Traditional Hosting Infrastructure and Services,Hosting (non-Cloud),Hosting (non-Cloud),Large Corporate (500-999),Core,ZAR,435.796428,474.704244,...,s8,1.539957e+03,1.555357e+03,1.570911e+03,1.354232e-02,1.460532e-02,Co-location and hosting,3.0,Mature,0.731878
